In [5]:
import pandas as pd
import numpy as np
import json
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import os

from CreatePlaylist import CreatePlaylist

In [6]:
# Load JSON data from file
with open('data/challenge_set.json', 'r') as file: # Replace with local dataset path
    data = json.load(file)

# Initialize an empty list to collect all track data
all_tracks = []

# Loop through each playlist in the dataset
for playlist in data['playlists']:
    for track in playlist['tracks']:
        # Add playlist-level information to each track record
        track_info = {
            'playlist_name': playlist.get('name', 'Unknown'),
            'playlist_pid': playlist['pid'],
            'playlist_num_tracks': playlist['num_tracks'],
            'track_pos': track['pos'],
            'artist_name': track['artist_name'],
            'track_uri': track['track_uri'],
            'artist_uri': track['artist_uri'],
            'track_name': track['track_name'],
            'album_uri': track['album_uri'],
            'duration_ms': track['duration_ms'],
            'album_name': track['album_name']
        }
        all_tracks.append(track_info)

# Convert the list of track dictionaries to a DataFrame
df_spotify = pd.DataFrame(all_tracks)

In [7]:
#df_get_genre = pd.read_csv('df_get_genre.csv')
df_get_genre = df_spotify
df_get_genre = df_get_genre.drop_duplicates(subset='artist_uri')['artist_uri'].to_frame()
df_get_genre['artist_genres'] = ""

artist_uris = df_get_genre['artist_uri'].unique().tolist()

In [8]:
spotify_api = CreatePlaylist()

In [13]:
def get_artist_genres(batch_size=50, count_batches=0):

    df_fetch_genres = pd.DataFrame(columns=['artist_uri', 'artist_genres'])

    start_index = int(input(count_batches))

    for i in range(start_index, len(artist_uris), batch_size):
        batch = artist_uris[i:i + batch_size]
        artist_genres_results = spotify_api.sp.artists(artists=batch)

        genres = []
        for i in artist_genres_results['artists']:
            genres.extend([i['genres']])
        zipped = list(zip(batch, genres))
        df_fetch_genres = pd.concat([df_fetch_genres, pd.DataFrame(zipped, columns=['artist_uri', 'artist_genres'])], ignore_index=True)

        # Write the resulting DataFrame to a new CSV file with semicolon as the delimiter
        df_fetch_genres.to_csv('tracks_with_genres.csv', sep=';', index=False)
        count_batches += 1
        print(f"Batches run successfully: {count_batches}", end='\r')
        last_batch = count_batches
    print('Done!')
    return df_fetch_genres

In [15]:
df_fetch_genres = get_artist_genres()

Done!es run successfully: 282


In [18]:
# Dummy encode artist_genre from their list structure
df_genre_dummies = pd.get_dummies(df_fetch_genres['artist_genres'].explode()).groupby(level=0).sum()
df_genre_dummies['artist_uri'] = df_fetch_genres['artist_uri']
df_genre_dummies.to_csv('dummy_encoded_genres.csv', index=False)